In [1]:
#@title Import Dependencies

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [11]:
#@title Define Hyperparameters

input_size = 784 # img_size = (28,28) ---> 28*28=784 in total
hidden_size = 20 # number of nodes at hidden layer
num_classes = 10 # number of output classes discrete range [0,9]
num_epochs = 20 # number of times which the entire dataset is passed throughout the model
batch_size = 100 # the size of input data took for one iteration
lr = 1e-3 # size of step 

In [3]:
#@title Downloading MNIST data

train_data = dsets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = dsets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

100%|██████████| 9912422/9912422 [00:00<00:00, 28402590.00it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 6406584.19it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 7325341.35it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 12057296.69it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [12]:
#@title Loading the data

train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size, 
                                      shuffle = False)

In [13]:
#@title Define model class

class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)
  
  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [17]:
#@title Build the model

net = Net(input_size, hidden_size, num_classes)
if torch.cuda.is_available():
  net.cuda()

In [18]:
#@title Define loss-function & optimizer

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam( net.parameters(), lr=lr)

In [19]:
#@title Training the model

for epoch in range(num_epochs):
  for i ,(images,labels) in enumerate(train_gen):
    images = Variable(images.view(-1,28*28)).cuda()
    labels = Variable(labels).cuda()
    
    optimizer.zero_grad()
    outputs = net(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()
    
    if (i+1) % 100 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.data))

Epoch [1/20], Step [100/600], Loss: 0.7586
Epoch [1/20], Step [200/600], Loss: 0.6578
Epoch [1/20], Step [300/600], Loss: 0.4132
Epoch [1/20], Step [400/600], Loss: 0.2653
Epoch [1/20], Step [500/600], Loss: 0.3760
Epoch [1/20], Step [600/600], Loss: 0.2321
Epoch [2/20], Step [100/600], Loss: 0.2010
Epoch [2/20], Step [200/600], Loss: 0.2502
Epoch [2/20], Step [300/600], Loss: 0.2265
Epoch [2/20], Step [400/600], Loss: 0.1894
Epoch [2/20], Step [500/600], Loss: 0.3196
Epoch [2/20], Step [600/600], Loss: 0.1802
Epoch [3/20], Step [100/600], Loss: 0.3340
Epoch [3/20], Step [200/600], Loss: 0.2048
Epoch [3/20], Step [300/600], Loss: 0.1734
Epoch [3/20], Step [400/600], Loss: 0.3398
Epoch [3/20], Step [500/600], Loss: 0.1892
Epoch [3/20], Step [600/600], Loss: 0.2637
Epoch [4/20], Step [100/600], Loss: 0.2435
Epoch [4/20], Step [200/600], Loss: 0.3686
Epoch [4/20], Step [300/600], Loss: 0.3716
Epoch [4/20], Step [400/600], Loss: 0.3624
Epoch [4/20], Step [500/600], Loss: 0.2705
Epoch [4/20

In [22]:
#@title Evaluating the accuracy of the model

correct = 0
total = 0
for images,labels in test_gen:
  images = Variable(images.view(-1,28*28)).cuda()
  labels = labels.cuda()
  
  output = net(images)
  _, predicted = torch.max(output,1)
  correct += (predicted == labels).sum()
  total += labels.size(0)

print('Accuracy of the model: %.3f %%' %((100*correct)/(total+1)))

Accuracy of the model: 96.100 %


In [21]:
print("Model's state_dict:")
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())

Model's state_dict:
fc1.weight 	 torch.Size([20, 784])
fc1.bias 	 torch.Size([20])
fc2.weight 	 torch.Size([10, 20])
fc2.bias 	 torch.Size([10])


In [23]:
torch.save(net.state_dict(), "../models/mnist/mnist-small.state_dict")